In [1]:
# mainly form the python file: Preprocessing_Germany_DAAD_4.ipynb (combined with the work Preprocessing_Germany_DAAD_2.ipynb )

import pandas as pd

encoding = 'utf-8'

In [3]:
# loading the bibliometric data


papers = pd.read_csv('N:\Bibliometric_Germany\KB data\scprp20_Ger_2_this_time_really_GER.csv',encoding=encoding)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
papers.columns

Index(['AUTHOR_ID', 'AFID', 'PK_INTITUTIONS', 'PK_ITEMS', 'PK_AUTHORS',
       'PUBYEAR', 'SOURCETITLE', 'DOI', 'source_id', 'PK_SOURCES',
       'ARTICLE_TITLE_EN', 'AUTHOR_ID_1', 'LAST_NAME', 'FIRST_NAME',
       'INDEXED_NAME', 'INITIALS', 'ORCID_ID', 'INSTITUTION_FULL',
       'POSTALCODE', 'CITY', 'COUNTRYCODE'],
      dtype='object')

In [ ]:
papers["COUNTRY_CODE"]=papers["COUNTRY_CODE"].apply(str)
papers["COUNTRY_CODE"]=papers["COUNTRY_CODE"].map(lambda x:x.upper())

# modify the wrong country code 
def substitue(x):
    if (x=="YUG") | (x=="YUX"): 
        x="SRB"
    elif (x=="ROM"):
        x="ROU"
    elif (x=="ANT"):
        x="NLD"
    elif (x=="DDR"):
        x="DEU"
    elif (x=="UMI"):
        x="USA"
    else:
        pass
    
    return x

papers_1["COUNTRYCODE"]=papers_1["COUNTRYCODE"].map(lambda x: substitue(x))
papers_2["COUNTRYCODE"]=papers_2["COUNTRYCODE"].map(lambda x: substitue(x))
Paper_aff["COUNTRY_CODE"]=Paper_aff["COUNTRY_CODE"].map(lambda x: substitue(x))

   ### 1. predict missing country 

In [ ]:
# records with country / records without country 
papers_Country=papers[papers['COUNTRYCODE'].notnull()]
papers_PredCountry=papers[papers['COUNTRYCODE'].isnull()]

In [ ]:
# samping records having country 

train_counrtry6_new=papers_Country.sample(n=1000000)
train_counrtry6_new.to_csv('N:\\Bibliometric_Germany\\KB data\\processed\\new_nov\\train_country6_new.csv',encoding=encoding)

In [ ]:
papers_Country.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_nov\\02_papers_Country.csv",encoding=encoding)
papers_PredCountry.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_nov\\02_papers_PredCountry.csv",encoding=encoding)

In [1]:
# predic country in Country_prediction_NeuralNetwork

In [ ]:
# read predicted records 
#papers_Country= pd.read_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_Feb\\0_papers_Country.csv",encoding=encoding)
papers_predCountry= pd.read_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_Feb\\0_predicted_reliable.csv",encoding=encoding)

   ### 2. suspicious authors & acquire the ultimate disambiguated records
  

In [ ]:
papers['number_of_countries'] = (papers.groupby('AUTHOR_ID')['COUNTRYCODE']).transform('nunique')
papers['number_of_publications'] = (papers.groupby('AUTHOR_ID')['PK_ITEMS']).transform('nunique')
papers["number_of_yearlyPub"]=papers['number_of_publications']/(papers.groupby('AUTHOR_ID')['PUBYEAR'].transform('nunique'))
papers

In [ ]:
# select DEU-realted authors

DEU_author=papers[papers["COUNTRYCODE"]=="DEU"]
papers_1=papers[papers["AUTHOR_ID"].isin(DEU_author['AUTHOR_ID'])]
papers=papers_1

In [3]:
# suspicious authors (> 6 countries or have publictions monthly)

suspicious=papers[(papers["number_of_countries"]>6) | \
                          (papers["number_of_publications"]>292)]
suspicious.to_csv('N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_Feb\\02_suspicious_records.csv',encoding=encoding)

non_suspicious=papers[~ papers['AUTHOR_ID'].isin(suspicious['AUTHOR_ID'])]


In [ ]:

disambiguated_1=pd.read_csv('N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_Feb\\02_suspicious_records_disambiguated_less_clusters_01.csv',encoding=encoding)

disambiguated_1.drop(columns={"Unnamed: 0","index","Unnamed: 0.1"},inplace=True)
disambiguated_1.rename(columns={"author_id_revised":"AUTHOR_ID_NEW"},inplace=True)


In [ ]:
# remove ineffective records with more than 10 clusters after disambiguations 

disambiguated_1=disambiguated_1[~(disambiguated_1["AUTHOR_ID"].isin(disambiguated_1[disambiguated_1["cluster1"]>10].AUTHOR_ID.tolist()))]

disambiguated_1=disambiguated_1[(disambiguated_1["PUBYEAR"]>1995)&(disambiguated_1["PUBYEAR"]<2021)]

In [ ]:
disambiguated_1['number_of_countries'] = (disambiguated_1.groupby('AUTHOR_ID_NEW')['COUNTRYCODE']).transform('nunique')
disambiguated_1['number_of_publications'] = (disambiguated_1.groupby('AUTHOR_ID_NEW')['PK_ITEMS']).transform('nunique')

In [ ]:
#save the ultimate records

disambiguated_1=disambiguated_1[(disambiguated_1["number_of_countries"]<=6)&(disambiguated_1["number_of_publications"]<=292)]

all_records=pd.concat([non_suspicious,disambiguated_1])
all_records.to_csv(all_records.to_csv('N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\03_all_records_disambiguated.csv',encoding=encoding))

### 3. ASJC code for each author


In [ ]:
#read the recode after disambigationg suspicious Author_ID
country_tuples=(all_records[['DOI','CODE','COUNTRYCODE','AUTHOR_ID_NEW','PUBYEAR']])
country_tuples['number_of_countries'] = (country_tuples.groupby('AUTHOR_ID_NEW')['COUNTRYCODE']).transform('nunique')

In [ ]:
country_tuples=country_tuples.replace({'CODE': {'multidiscipl': 1000,'1200;':1200,'1300;':1300,'2800;':2800}})
country_tuples=country_tuples.astype({'CODE':int})
country_tuples["CODE"].unique()

In [ ]:
# Changing subject codes to the two left-most digits (for the main categories)

import numpy as np


country_tuples['CODE_1']=country_tuples['CODE'].apply(lambda x: np.floor(x/100))
country_tuples["CODE_2"]=country_tuples["CODE_1"]
country_tuples=country_tuples[(country_tuples["CODE_2"]!=81)&(country_tuples["CODE_2"]!=261)]

In [ ]:
# code dictionary (two scales)

dic_1 = {
10: "Multidisciplinary",
12: "Social",
14: "Social",
18: "Social",
20: "Social",
32: "Social",
33: "Social",
15: "Physical",
16: "Physical",
17: "Physical",
19: "Physical",
21: "Physical",
22: "Physical",
23: "Physical",
25: "Physical",
26: "Physical",
31: "Physical",
11: "Life",
13: "Life",
24: "Life",
28: "Life",
30: "Life",
27: "Health",
29: "Health",
34: "Health",
35: "Health",
36: "Health"
     }
dic_2 = {
10: "Multidisciplinary",
12: "Arts&Humanities",
14: "Business&Management&Accounting",
18: "DecisionSciences",
20: "Economics&Econometrics&Finance",
32: "Psychology",
33: "SocialSciences",
15: "ChemicalEngineering",
16: "Chemistry",
17: "ComputerScience",
19: "Earth&PlanetarySciences",
21: "Energy",
22: "Engineering",
23: "EnvironmentalScience",
25: "MaterialsScience",
26: "Mathematics",
31: "Physics&Astronomy",
11: "Agricultural&BiologicalSciences",
13: "Biochemistry&Genetics&MolecularBiology",
24: "Immunology&Microbiology",
28: "Neuroscience",
30: "Pharmacology&Toxicology&Pharmaceutics",
27: "Medicine",
29: "Nursing",
34: "veterinary",
35: "Dentistry",
36: "HealthProfessions"
     }

country_tuples=country_tuples.replace({"CODE_1": dic_1})
country_tuples=country_tuples.replace({"CODE_2": dic_2})
country_tuples.drop_duplicates(inplace=True)

In [ ]:
# for finer scale (dic_2)

percentage=(country_tuples.groupby('AUTHOR_ID_NEW')['CODE_2'].value_counts(normalize=True).to_frame())
percentage=percentage.rename(columns={"CODE_2": "FREQ"})
percentage=percentage.reset_index()

In [ ]:
num=percentage["AUTHOR_ID_NEW"].nunique()
freq_mean={}
freq_std={}
codearray=percentage["CODE_2"].unique()
codelen=percentage["CODE_2"].nunique()
for i in range(0,codelen):
    index=codearray[i]
    mean=sum((percentage[percentage["CODE_2"]==index])["FREQ"])/num
    std=sum((percentage[percentage["CODE_2"]==index])["FREQ"].apply(lambda x: (x-mean)*(x-mean)))/num
    #mean=(percentage[percentage["CODE_2"]==index])["FREQ"].mean()
    #std=(percentage[percentage["CODE_2"]==index])["FREQ"].std()
    freq_mean[index]=mean
    freq_std[index]=std**0.5
    #freq_std[index]=std
print(freq_mean)
print(freq_std)

In [ ]:
# alpha is chosen by trial and error such that the fraction of authors for whome disciplines are inferred 
# would be above 89%

alpha=0.25

index=0
author_discipline={}
for identity, row in percentage.groupby("AUTHOR_ID_NEW"):
    #print(identity)  
    index=row["FREQ"].index[0]
    code=row["CODE_2"][index]
    significant_index=0
    significant_score=0
    for i in range(len(row)):
        z_score=(list(row["FREQ"])[i]-freq_mean[list(row["CODE_2"])[i]])/(freq_std[list(row["CODE_2"])[i]])
        if z_score > alpha:
            if z_score>significant_score:
                significant_index=i
                significant_score=z_score
    if significant_score>alpha:
        author_discipline[identity]= list(row["CODE_2"])[significant_index]

In [ ]:
# "mapping" the disctionary back into the dataframe to create the column "MAIN_CODE"

country_tuples_code['MAIN_CODE'] = country_tuples_code['AUTHOR_ID_NEW'].map(author_discipline)

# add Multidis for missing values of MAIN_CODE
country_tuples_code["MAIN_CODE"].fillna("Multidisciplinary", inplace=True)

# select the valid records again, filter invaild records

tuples=country_tuples.copy()
country_tuples_unique=country_tuples.drop_duplicates(['AUTHOR_ID_NEW','PK_ITEMS']).sort_values(by=['AUTHOR_ID_NEW','PUBYEAR'])
country_tuples_unique['number_of_countries'] = (country_tuples_unique.groupby('AUTHOR_ID_NEW')['COUNTRYCODE']).transform('nunique')
country_tuples_unique["PUBLICATION_COUNT"] = (country_tuples_unique.groupby('AUTHOR_ID_NEW')['PK_ITEMS']).transform('nunique')

country_tuples_unique=country_tuples_unique[country_tuples_unique["number_of_countries"]<5]
country_tuples_unique=country_tuples_unique[country_tuples_unique["PUBLICATION_COUNT"]<293]


In [ ]:
# save all the authors including movers and non-movers
country_tuples_code.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\05_AuthorID_Main_Code_old_method.csv",encoding=encoding)

### 4. NLP for main disciplines over publications

In [ ]:
# Load key words
Key_word= pd.read_csv("N:\Bibliometrics\data\scopus_raw_2020\scopusrp20_utf8_keywords.csv",encoding=encoding)
Key_word['KEYWORD']=Key_word['KEYWORD'].astype("str")
Key_word.drop(columns={"KEYWORD_TYPE","PK_KEYWORDS"},inplace=True)

Key_word.rename(columns={'FK_ITEMS':'PK_ITEMS'}, inplace=True)
Key_word_new=Key_word.groupby(by='PK_ITEMS').apply(lambda x:' '.join(x['KEYWORD'])) 
Key_word_new=Key_word_new.to_frame().reset_index()
Key_word_new.rename(columns={0:'KEYWORD'},inplace=True)

In [ ]:
# preparing for LDA
# text source: Title + Journal title + Keyword 

all_records['ARTICLE_TITLE_EN']=all_records['ARTICLE_TITLE_EN'].fillna(" ")
all_records['SOURCETITLE']=all_records['SOURCETITLE'].fillna("")
all_records['KEYWORD']=all_records['KEYWORD'].fillna(" ")
all_records["Text"]= all_records['ARTICLE_TITLE_EN']+" "+all_records['SOURCETITLE']+" "+all_records['KEYWORD']

# keep only text 
author_text=all_records[["AUTHOR_ID_NEW","Text"]]
author_text.drop_duplicates(inplace=True)
author_text=author_text.groupby(by='AUTHOR_ID_NEW').apply(lambda x:' '.join(x['Text']))
author_text=author_text.to_frame().reset_index()
author_text.rename(columns={0:'Text'},inplace=True)

#save to file
author_text.to_csv('N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\04_all_records_Text.csv',encoding=encoding)

### 5. Citation data 

In [ ]:
# load citation data

citations=pd.read_csv("N:\\Theile\\bibliometry\\scopus_rp_2020\\scopus_rp_2020_all_citingcount.csv",encoding=encoding)
citations=citations.rename(columns={"RP2020PK_ITEMS":"PK_ITEMS"})
citations=citations.drop(columns={"UT_EID"})
publications=pd.merge(country_tuples_unique,citations,how='left',on="PK_ITEMS")
publications=publications.drop_duplicates(['AUTHOR_ID_NEW','PK_ITEMS']).sort_values(by=['AUTHOR_ID_NEW','PUBYEAR'])


all_records_2['TOTAL_CITATIONS']=all_records_2.groupby(['AUTHOR_ID_NEW'])['COUNT'].transform('sum')
all_records_2["AVERAGE_CITATIIONS_perpub"]=(all_records_2["TOTAL_CITATIONS"]/all_records_2["PUBLICATION_COUNT"])

all_records_2['min_year'] = (all_records_2.groupby('AUTHOR_ID_NEW')['PUBYEAR']).transform('min')
all_records_2['ACADEMIC_AGE'] = (2020-all_records_2['min_year']+1)
all_records_2=all_records_2.drop('min_year', axis=1)

all_records_2["AVERAGE_CITATIIONS_peryear"]=(all_records_2["TOTAL_CITATIONS"]/all_records_2["ACADEMIC_AGE"])
all_records_2=all_records_2.rename(columns={"COUNT":"CITATION"})

all_records_2.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\06_AuthorID_Citation_Age.csv",encoding=encoding)

### 6. Migration detection 

In [ ]:
country_tuples['min_year'] = (country_tuples.groupby('AUTHOR_ID_NEW')['PUBYEAR']).transform('min')
country_tuples['number_of_countries'] = (country_tuples.groupby('AUTHOR_ID_NEW')['COUNTRYCODE']).transform('nunique')
country_tuples["PUBLICATION_COUNT"] = (country_tuples.groupby('AUTHOR_ID_NEW')['PK_ITEMS']).transform('nunique')
country_tuples['ACADEMIC_AGE'] = (2020-country_tuples['min_year']+1)
country_tuples=country_tuples.drop('min_year', axis=1)



In [ ]:
country_tuples["PUBYEAR_sort"]=country_tuples.groupby("AUTHOR_ID_NEW")["PUBYEAR"].rank(ascending=1,method='dense')
country_tuples["PUBYEAR_sort_rev"]=country_tuples.groupby("AUTHOR_ID_NEW")["PUBYEAR"].rank(ascending=0,method='dense')
country_tuples=country_tuples.sort_values(by=['AUTHOR_ID_NEW','PUBYEAR',"PK_ITEMS"])
country_tuples['No_Country_perpub'] = (country_tuples.groupby(['AUTHOR_ID_NEW',"PK_ITEMS"])['COUNTRYCODE']).transform('nunique')
country_tuples['No_Country_peryear'] = (country_tuples.groupby(['AUTHOR_ID_NEW',"PUBYEAR"])['COUNTRYCODE']).transform('nunique')

country_tuples=country_tuples.sort_values(by=['AUTHOR_ID_NEW','PUBYEAR',"PK_ITEMS"])

In [ ]:
# for determining the origin and destination countries if the first year and last year covers multiple countries
#
# modication version, but origin and destination will still be modifed in the next steps 

#****************************************************************
from scipy import stats

country_tuples["DESTINATION"]="destination"
country_tuples["containDEU"]=2
origin_list={}
destination_list={}
containDEU={}
for identity, row in country_tuples.groupby("AUTHOR_ID_NEW"):
    if row['COUNTRYCODE'].str.contains('DEU').any():
        DEU=1
    else:
        DEU=0
    
    #if (row["No_Country_perpub"].iloc[0]>1) & ((row["PUBYEAR_sort"].iloc[0])!=(row["PUBYEAR_sort_rev"].iloc[0])):
    
    if (row["No_Country_peryear"].iloc[0]>1):
        origin=row[row["PUBYEAR_sort"]==1].COUNTRYCODE.mode().iloc[0]
        
    else:
        origin=row["COUNTRYCODE"].iloc[0]
        
    #f (row["No_Country_perpub"].iloc[-1]>1) & ((row["PUBYEAR_sort"].iloc[-1])!=(row["PUBYEAR_sort_rev"].iloc[-1])):
    if (row["No_Country_peryear"].iloc[-1]>1):
        destination=row[row["PUBYEAR_sort_rev"]==1].COUNTRYCODE.mode().iloc[0]
    else: 
        destination=row["COUNTRYCODE"].iloc[-1] 
        
    if (row["ACADEMIC_AGE"].iloc[0]==1):
        origin=row.COUNTRYCODE.mode().iloc[0]
        destination=row.COUNTRYCODE.mode().iloc[0]
    else: 
        pass          
        
    
    origin_list[identity]= origin
    destination_list[identity]= destination
    containDEU[identity]= DEU
        
country_tuples["ORIGIN"]=country_tuples['AUTHOR_ID_NEW'].map(origin_list)
country_tuples["DESTINATION"]=country_tuples['AUTHOR_ID_NEW'].map(destination_list)   
country_tuples["containDEU"]=country_tuples['AUTHOR_ID_NEW'].map(containDEU) 


In [ ]:
# backup
country_tuples_ori=country_tuples.copy()
country_tuples=country_tuples[country_tuples["containDEU"]==1]
country_tuples.drop(columns={"No_Country_perpub","containDEU"},inplace=True)

country_tuples['number_of_countries'] = (country_tuples.groupby('AUTHOR_ID_NEW')['COUNTRYCODE']).transform('nunique')
country_tuples["PUBLICATION_COUNT"] = (country_tuples.groupby('AUTHOR_ID_NEW')['PK_ITEMS']).transform('nunique')

In [ ]:
# non-movers: 
non_movers=country_tuples[(country_tuples["number_of_countries"]==1)]



In [ ]:
# mode country for each year

# mode for countrycode
locations_by_year=country_tuples[['AUTHOR_ID_NEW','PUBYEAR','COUNTRYCODE',"ORIGIN","DESTINATION"]].groupby(['AUTHOR_ID_NEW','PUBYEAR'])['COUNTRYCODE'].agg(pd.Series.mode).to_frame()

# sum for countrycode
#locations_by_year=population.groupby(['AUTHOR_ID_NEW','PUBYEAR'])['COUNTRYCODE'].agg(pd.Series.sum).to_frame()

#Resetting the index
locations_by_year=pd.DataFrame(locations_by_year).reset_index()
# Tom's functions work if the countrycode is a string as opposed to an array when it is multiple mode
locations_by_year=locations_by_year.astype({'COUNTRYCODE':str})
locations_by_year.head(20)

In [ ]:
import numpy as np
locations_by_year["AUTHOR_ID_NEW_NEXT"]=locations_by_year.AUTHOR_ID_NEW.shift(-1)
locations_by_year["AUTHOR_ID_NEW_LAST"]=locations_by_year.AUTHOR_ID_NEW.shift(1)
#locations_by_year["COUNTRYCODE_NEXT"]=locations_by_year.apply(lambda row:row.COUNTRYCODE.shift(-1) if(row.AUTHOR_ID_NEW==row.AUTHOR_ID_NEW_NEXT) else np.NaN)
locations_by_year['COUNTRYCODE_NEXT'] = locations_by_year.COUNTRYCODE.shift(-1)
locations_by_year['COUNTRYCODE_LAST'] = locations_by_year.COUNTRYCODE.shift(1)

#null value , np.nan
locations_by_year.loc[locations_by_year['AUTHOR_ID_NEW'] != locations_by_year['AUTHOR_ID_NEW_NEXT'], 'COUNTRYCODE_NEXT'] = np.NaN
locations_by_year.loc[locations_by_year['AUTHOR_ID_NEW'] != locations_by_year['AUTHOR_ID_NEW_LAST'], 'COUNTRYCODE_LAST'] = np.NaN            

locations_by_year['number_of_countries_new'] = (locations_by_year.groupby('AUTHOR_ID_NEW')['COUNTRYCODE']).transform('nunique')

In [ ]:
author_records=country_tuples.drop_duplicates()
author_records_1=author_records[["AUTHOR_ID_NEW","PUBLICATION_COUNT","ACADEMIC_AGE",'ORIGIN', 'DESTINATION']]
author_mergeLocation=pd.merge(locations_by_year,author_records_1,how='left',on="AUTHOR_ID_NEW") 
#author_mergeLocation.rename(columns={'COUNTRYCODE_x':'COUNTRYCODE',"COUNTRYCODE_y":"locations_by_year"},inplace=True)                                

#author_mergeLocationgeonames_admin1_code
#author_mergeLocation["number_of_countries"]=(author_mergeLocation.groupby('AUTHOR_ID_NEW')['COUNTRYCODE']).transform('nunique')

author_mergeLocation.drop_duplicates(["AUTHOR_ID_NEW","PUBYEAR"],inplace=True)

author_mergeLocation.to_csv('N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\07_Location_by_year_useful.csv', index = False)

In [ ]:
author_mergeLocation.rename(columns={"number_of_countries_new":"number_of_countries"},inplace=True)
moving_author=author_mergeLocation[(author_mergeLocation["number_of_countries"]>1) & (author_mergeLocation["PUBLICATION_COUNT"]>1) & (author_mergeLocation["ACADEMIC_AGE"]>1)]
non_moving_author=author_mergeLocation[(author_mergeLocation["number_of_countries"]==1) | (author_mergeLocation["PUBLICATION_COUNT"]==1)| (author_mergeLocation["ACADEMIC_AGE"]==1)]

In [ ]:
# for multiple modes value 

from random import randint
from collections import Counter
def mode_list(li):
    
    mode= [k for k,v in Counter(li).items() if v == max(Counter(li).values())]
    
    if (len(mode)==1):
        mode_str=''.join(mode)
    else:
        mode_str=mode[randint(0,len(mode)-1)]
    
    return mode_str
bb=['BEL', 'DEU']
mode_list(bb)   

In [ ]:
from collections import Set
from random import randint
import re
# Creating country of residence for all records (next chunck)

%time
#%debug
residenceCountry = None
ORIGIN_NEW = None
lastyear = None

newtable = []
newrow = []

authorid = -1
moving_author.sort_values(by=['AUTHOR_ID_NEW','PUBYEAR'],inplace=True)
for row in moving_author.itertuples():
    #print(row)
    newrow = row._asdict()
    newrow_1 = row._asdict()
    if row.AUTHOR_ID_NEW != authorid:
        #print("------------------------------------new auhtor!!")
        #new author
        residenceCountry = None
        ORIGIN_NEW = None
        lastyear = None
        i=1
    authorid = row.AUTHOR_ID_NEW
    
    #print(residenceCountry)
    #first country: take the first mode
    # 1. first year--multiple countries 
    #    1.1 2nd year--1 country 
    #        1.1.1 overlap with 1st year: remove the overaped countries (??), mode
    #        1.1.2 overlap with 1st year: mode
    #    1.2 2nd year-- > 1 countries
    #        ---union of 1st year and 2nd year
    #        1.2.1 union>=1:  mode (union) 
    #        1.2.2 union>=0:  mode (1st year)
    # 2. last year--multiple countries 
    # combine with last year, mode()    
#***************************************************************
#new thougts (oct 21)
    # 1. first year--multiple countries 
    #    union with second year
    #    1.1 union>=1: mode(union) 
    #    1.2 union==0: mode(1st year) 
    # 2. last year--multiple countries 
    #    union with (last-1) year
    #    1.1 union>=1: mode(union) 
    #    1.2 union==0: mode(last year) 


    if not residenceCountry:
        if ((len(row.COUNTRYCODE)>3) ):
            # remove ",", '[]'
            xx=re.findall("'([A-Z]+)'",row.COUNTRYCODE)
            if len(row.COUNTRYCODE_NEXT)==3:
                
                if any(row.COUNTRYCODE_NEXT in s for s in xx):
                    xx.remove(row.COUNTRYCODE_NEXT)
                    newrow["residenceCountry"] = mode_list(xx)
                    residenceCountry = newrow["residenceCountry"]
                else: 
                    newrow["residenceCountry"] = xx[randint(0,len(xx)-1)]
                    residenceCountry = newrow["residenceCountry"]
                    
                ORIGIN_NEW=newrow["residenceCountry"]
                newrow["ORIGIN_NEW"]=ORIGIN_NEW   
                    
            else:
                yy=re.findall("'([A-Z]+)'",row.COUNTRYCODE_NEXT)
                union=list(set(xx)&(set(yy)))
                #yy=list(set(xx)-(set(xx)&set(yy)))
                if len(union)>0: 
                    newrow["residenceCountry"] = union[randint(0,len(union)-1)]
                    residenceCountry = newrow["residenceCountry"]
                    
                else:
                    newrow["residenceCountry"] = mode_list(xx)
                    residenceCountry = newrow["residenceCountry"]
                    
                ORIGIN_NEW=newrow["residenceCountry"]
                newrow["ORIGIN_NEW"]=ORIGIN_NEW   
                
        else:
            newrow["residenceCountry"]=row.COUNTRYCODE
            residenceCountry = newrow["residenceCountry"]
            ORIGIN_NEW=newrow["residenceCountry"]
            newrow["ORIGIN_NEW"]=ORIGIN_NEW    

        
    elif row.AUTHOR_ID_NEW != row.AUTHOR_ID_NEW_NEXT:    
        if (len(row.COUNTRYCODE)>3):
            xx=re.findall("'([A-Z]+)'",row.COUNTRYCODE)
            if len(row.COUNTRYCODE_LAST)>3:
                xx.extend(re.findall("'([A-Z]+)'",row.COUNTRYCODE_LAST))
                newrow["residenceCountry"] = mode_list(xx)
            else: 
                xx.append(row.COUNTRYCODE_LAST)
                newrow["residenceCountry"] = mode_list(xx)
        else:
            newrow["residenceCountry"]=row.COUNTRYCODE 
                   
        residenceCountry = None
        DESTINATION_NEW=newrow["residenceCountry"]
        newrow["ORIGIN_NEW"]=ORIGIN_NEW
        newrow["DESTINANTION_NEW"]=DESTINATION_NEW

    elif len(row.COUNTRYCODE)>3:
        # there is already a residenceCountry and we have multiple countries
        modes = re.findall("'([A-Z]+)'",row.COUNTRYCODE)
        
        if len(row.COUNTRYCODE_NEXT)>3:
            modes.extend(re.findall("'([A-Z]+)'",row.COUNTRYCODE_NEXT))
            newrow["residenceCountry"] = mode_list(modes)
        else:
            modes.append(row.COUNTRYCODE_NEXT)
            newrow["residenceCountry"] = mode_list(modes)
        
        residenceCountry = row.COUNTRYCODE
        newrow["ORIGIN_NEW"]=ORIGIN_NEW
            
        
    else:
        # we have only one mode -- take that!
        #print("sdf3- only one mode!", residenceCountry,row.COUNTRYCODE)
        newrow["residenceCountry"] = row.COUNTRYCODE
        residenceCountry=row.COUNTRYCODE
        newrow["ORIGIN_NEW"]=ORIGIN_NEW
        
        


        
    #print("-------------",newrow)
    newtable.append(newrow)
    #newtable.append(newrow_1)
    
dfcorrected = pd.DataFrame(newtable)
dfcorrected['DESTINATION_NEW']=dfcorrected.groupby(['AUTHOR_ID_NEW'])['DESTINANTION_NEW'].apply(lambda x: x.fillna(x.mode()[0]))

In [ ]:
dfcorrected.drop(columns={"COUNTRYCODE_NEXT","AUTHOR_ID_NEW_NEXT","ORIGIN","DESTINATION","COUNTRYCODE_LAST","AUTHOR_ID_NEW_LAST"},inplace=True)
dfcorrected.rename(columns={"ORIGIN_NEW":"ORIGIN"},inplace=True)
dfcorrected.rename(columns={"DESTINATION_NEW":"DESTINATION"},inplace=True)
dfcorrected['number_of_countries'] = (dfcorrected.groupby('AUTHOR_ID_NEW')['residenceCountry']).transform('nunique')
dfcorrected['min_year'] = (dfcorrected.groupby('AUTHOR_ID_NEW')['PUBYEAR']).transform('min')

dfcorrected.to_csv('N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\07_new_Location_by_year_mover_corrected.csv', index = False)

In [ ]:
#Display migration event 

from math import floor,ceil

def get_migrationevents_from_dfcorrected(dfcorrected):
    migrationtable = []
    inmigration = []
    outmigration = [] #(country, year, )
    population = []

    authorid=-1
    lastcountry = None
    lastyear= None
    thisyearcountry = None
    for row in dfcorrected.itertuples():
        #print(row)#newrow=
        newrow = row._asdict()
        year = row.PUBYEAR

        if row.AUTHOR_ID_NEW != authorid:
            #print("------------------------------------new auhtor!!")
            #new author
            residenceCountry = None
            thisyearcountry = None
            lastyear = None
            lastcountry=None
            authorid = row.AUTHOR_ID_NEW

        thisyearcountry = row.residenceCountry
        population.append((thisyearcountry, year, 1))
        if  (lastcountry != thisyearcountry):  # (pd.notnull(lastcountry)) and
            #migration event!
            #print("Migration-Event!", lastcountry,thisyearcountry)
            if lastcountry:
                migrationyear = ceil((lastyear+year)/2)
                newrow["from"] = lastcountry 
                newrow["to"] = thisyearcountry
                
                newrow["move_year"] = int(ceil((lastyear+year)/2))
                inmigration.append((thisyearcountry, migrationyear))
                outmigration.append((lastcountry,migrationyear))
                newrow["migrationevents"] =  lastcountry+"|" + thisyearcountry
            else:
                newrow["migrationevents"] =  "nnn|" + thisyearcountry
        lastcountry = row.residenceCountry
        lastyear=year
        migrationtable.append(newrow)

    dfmigrationevents = pd.DataFrame(migrationtable)
    dfinmigration = pd.DataFrame(inmigration)
    dfoutmigration = pd.DataFrame(outmigration)
    dfpopulation = pd.DataFrame(population)
    return dfmigrationevents,dfinmigration, dfoutmigration, dfpopulation


dfmigrationevents,dfinmigration, dfoutmigration, dfpopulation = get_migrationevents_from_dfcorrected(dfcorrected)
dfmigrationevents=dfmigrationevents.drop(columns=['Index', '_1'])

dfmigrationevents=dfmigrationevents.groupby("AUTHOR_ID_NEW").filter(lambda g: any(g.residenceCountry == 'DEU')) 
dfmigrationevents=dfmigrationevents[dfmigrationevents["number_of_countries"]>1]

fmigrationevents.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\08_new1_migrationevents_from_to_moveyear_all.csv")

In [ ]:
# foe non-movers
non_moving_author['min_year'] = (non_moving_author.groupby('AUTHOR_ID_NEW')['PUBYEAR']).transform('min')
non_moving_author['max_year'] = (non_moving_author.groupby('AUTHOR_ID_NEW')['PUBYEAR']).transform('max')

non_moving_author.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\08_new_non_movers_Germany.csv")

In [ ]:
# Result: different migration types

print(dfmigrationevents.AUTHOR_ID_NEW.nunique())
print(dfmigrationevents[(dfmigrationevents["ORIGIN"]=="DEU")&(dfmigrationevents["DESTINATION"]!="DEU")].AUTHOR_ID_NEW.nunique())
print(dfmigrationevents[(dfmigrationevents["ORIGIN"]!="DEU")&(dfmigrationevents["DESTINATION"]=="DEU")].AUTHOR_ID_NEW.nunique())
print(dfmigrationevents[(dfmigrationevents["ORIGIN"]=="DEU")&(dfmigrationevents["DESTINATION"]=="DEU")].AUTHOR_ID_NEW.nunique())
print(dfmigrationevents[(dfmigrationevents["ORIGIN"]!="DEU")&(dfmigrationevents["DESTINATION"]!="DEU")].AUTHOR_ID_NEW.nunique())